In [7]:
import pandas as pd
import numpy as np
# from tqdm import tqdm # Закомментировано для совместимости, если tqdm не установлен

def merge_intervals_with_gap(intervals, gap_threshold=0.5):
    """
    Объединяет перекрывающиеся или близко расположенные интервалы на основе порогового значения зазора.

    Args:
        intervals: Список интервалов в виде кортежей (начало, конец).
        gap_threshold: Максимально допустимый зазор между интервалами для объединения.

    Returns:
        Список объединенных интервалов.
    """
    if not intervals:
        return []

    # Сортируем интервалы по времени начала
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        prev_start, prev_end = merged[-1]

        # Проверяем перекрытие или попадание зазора в пороговое значение
        if current_start <= prev_end or (current_start - prev_end) <= gap_threshold:
            # Объединяем интервалы
            merged[-1] = (prev_start, max(prev_end, current_end))
        else:
            # Добавляем как новый интервал
            merged.append((current_start, current_end))

    return merged

def get_intersection_of_merged_intervals(intervals1, intervals2):
    """
    Вычисляет общую длительность пересечения между двумя списками *уже объединенных* интервалов.

    Args:
        intervals1: Список объединенных интервалов в виде кортежей (начало, конец).
        intervals2: Список объединенных интервалов в виде кортежей (начало, конец).

    Returns:
        Общая длительность пересечения.
    """
    total_intersection = 0
    i = 0
    j = 0

    while i < len(intervals1) and j < len(intervals2):
        # Берем текущие интервалы
        int1_start, int1_end = intervals1[i]
        int2_start, int2_end = intervals2[j]

        # Находим перекрытие между текущими интервалами
        overlap_start = max(int1_start, int2_start)
        overlap_end = min(int1_end, int2_end)
        overlap_duration = max(0, overlap_end - overlap_start)

        total_intersection += overlap_duration

        # Переходим к следующему интервалу в списке, который заканчивается раньше
        if int1_end < int2_end:
            i += 1
        else:
            j += 1

    return total_intersection


def evaluate_metrics_area_based(true_segments, pred_segments, merge_gap_true=0.0, merge_gap_pred=0.0):
    """
    Оценивает метрики на основе общей длительности (площади) интервалов.

    Args:
        true_segments: Список истинных интервалов (начало, конец).
        pred_segments: Список предсказанных интервалов (начало, конец).
        merge_gap_true: Порог зазора для объединения истинных интервалов перед расчетом площади.
        merge_gap_pred: Порог зазора для объединения предсказанных интервалов перед расчетом площади.


    Returns:
        Словарь вычисленных метрик (Precision_Area, Recall_Area, F1-score_Area).
    """
    # Объединяем интервалы внутри каждого набора, чтобы вычислить общую неперекрывающуюся площадь
    merged_true_segments = merge_intervals_with_gap(true_segments, gap_threshold=merge_gap_true)
    merged_pred_segments = merge_intervals_with_gap(pred_segments, gap_threshold=merge_gap_pred)
    print(merged_true_segments, "\n", merged_pred_segments)
    # Вычисляем общие площади
    total_true_area = sum(end - start for start, end in merged_true_segments)
    total_predicted_area = sum(end - start for start, end in merged_pred_segments)

    # Вычисляем площадь пересечения между двумя объединенными наборами
    intersection_area = get_intersection_of_merged_intervals(merged_true_segments, merged_pred_segments)

    # Вычисляем метрики на основе площадей
    precision_area = intersection_area / total_predicted_area if total_predicted_area > 0 else 0.0
    recall_area = intersection_area / total_true_area if total_true_area > 0 else 0.0
    f1_score_area = 2 * (precision_area * recall_area) / (precision_area + recall_area) if (precision_area + recall_area) > 0 else 0.0

    print(f"Общая истинная площадь: {total_true_area:.4f}, Общая предсказанная площадь: {total_predicted_area:.4f}, Площадь пересечения: {intersection_area:.4f}")

    return {
        'Precision_Area': precision_area,
        'Recall_Area': recall_area,
        'F1-score_Area': f1_score_area
    }


def bootstrap_confidence_intervals_area_based(true_segments, pred_segments, merge_gap_true=0.0, merge_gap_pred=0.0,
                                  n_bootstrap=1000, confidence_level=0.95):
    """
    Вычисляет бутстрап доверительные интервалы с использованием метрик оценки на основе площадей.
    """
    n_true = len(true_segments)
    n_pred = len(pred_segments)

    metrics_samples = {
        'Precision_Area': [],
        'Recall_Area': [],
        'F1-score_Area': []
    }

    # Используем простой цикл range вместо tqdm для совместимости
    for _ in range(n_bootstrap):
        # Бутстрап с заменой из исходных выборок
        true_bootstrap_indices = np.random.choice(n_true, n_true, replace=True)
        pred_bootstrap_indices = np.random.choice(n_pred, n_pred, replace=True)

        true_bootstrap = [true_segments[i] for i in true_bootstrap_indices]
        pred_bootstrap = [pred_segments[i] for i in pred_bootstrap_indices]

        # Используем функцию оценки метрик на основе площадей
        metrics = evaluate_metrics_area_based(true_bootstrap, pred_bootstrap, merge_gap_true=merge_gap_for_true_area, merge_gap_pred=merge_gap_for_pred_area)
        for k in metrics_samples:
            metrics_samples[k].append(metrics[k])

    # Вычисление квантилей для всех метрик
    alpha = (1 - confidence_level) / 2
    ci = {}

    for metric, samples in metrics_samples.items():
        ci[metric + "_CI"] = np.percentile(samples, [alpha*100, (1-alpha)*100])

    return ci

# --- Пример использования с метриками на основе площадей ---

# Убедитесь, что ваши файлы 'detections.csv' и 'true_label.txt' находятся в директории '../dataset/'
# относительно места запуска скрипта, или обновите пути к файлам ниже.
try:
    # Загрузка предсказаний
    detections = pd.read_csv('../dataset/detections.csv')
    # Фильтруем по метке 'call' и извлекаем интервалы
    detections_call = detections[detections['label'] == 'call']
    predicted_intervals = list(zip(detections_call['start_s'], detections_call['end_s']))

    # Загрузка истинных меток
    true_intervals = []
    with open('../dataset/true_label.txt', 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) < 2:
                continue
            start_str = parts[0].replace(',', '.')
            end_str = parts[1].replace(',', '.')
            start = float(start_str)
            end = float(end_str)
            true_intervals.append((start, end))

    # --- Настройка для метрик на основе площадей ---
    # Пороги зазора для объединения интервалов *перед* расчетом общей площади и пересечения.
    # Порог 0.0 означает объединение только перекрывающихся/касающихся интервалов.
    # Можно установить небольшой порог, если, например, очень короткие паузы между истинными звуками
    # должны считаться частью одной "площади".
    merge_gap_for_true_area = 0
    merge_gap_for_pred_area = 0 # Используйте 0.0 или небольшой порог, соответствующий тому,
                                   # как вы хотите, чтобы предсказания "схлопывались" для площади

    # Оценка метрик на основе площадей
    metrics_area_based = evaluate_metrics_area_based(true_intervals, predicted_intervals,
                                                     merge_gap_true=merge_gap_for_true_area,
                                                     merge_gap_pred=merge_gap_for_pred_area)

    print("\nМетрики (оценка на основе площадей):")
    for metric, value in metrics_area_based.items():
        print(f"{metric}: {value:.4f}")

    # Расчет бутстрап доверительных интервалов для метрик на основе площадей
    # Установите n_bootstrap на меньшее число, например 100, для более быстрого выполнения, если необходимо.
    bootstrap_ci_area_based = bootstrap_confidence_intervals_area_based(true_intervals, predicted_intervals,
                                                                        merge_gap_true=merge_gap_for_true_area,
                                                                        merge_gap_pred=merge_gap_for_pred_area,
                                                                        n_bootstrap=1000) # Использование 100 бутстрап выборок для примера

    print("\nДоверительные интервалы (оценка на основе площадей):")
    for metric, ci_values in bootstrap_ci_area_based.items():
        print(f"{metric}: [{ci_values[0]:.4f} {ci_values[1]:.4f}]")

except FileNotFoundError:
    print("Ошибка: файлы датасета не найдены. Убедитесь, что 'detections.csv' и 'true_label.txt' находятся в директории '../dataset/'.")
except Exception as e:
    print(f"Произошла ошибка: {e}")

[(16.64595, 17.64888), (24.368349, 25.349689), (41.309021, 42.06636), (42.17569, 42.778351), (43.80769, 44.47702), (49.790119, 50.428871), (51.514622, 52.214931), (61.05003, 61.72472), (75.898003, 77.025993), (84.827797, 85.286003), (109.514198, 110.776497), (116.905998, 117.618103), (128.7603, 129.434601), (131.129303, 131.864105), (134.044998, 134.736801), (157.111206, 157.822998), (158.680695, 159.958206), (160.797699, 161.746704), (167.8526, 168.423004), (172.183502, 172.779694), (186.304596, 187.077194), (187.624695, 188.518906), (189.139404, 189.723404), (198.684097, 199.358398), (203.811203, 204.407898), (222.542496, 223.023102), (252.985703, 253.841599), (254.650299, 255.349899), (279.752411, 280.257294)] 
 [(16.5, 18.0), (24.0, 25.5), (41.0, 45.0), (48.5, 50.5), (51.0, 52.5), (60.0, 62.0), (75.5, 77.5), (84.0, 85.5), (92.5, 93.5), (99.0, 100.0), (103.5, 106.0), (109.0, 111.0), (116.5, 118.0), (128.5, 129.5), (130.5, 132.0), (133.5, 135.0), (157.0, 158.0), (158.5, 159.5), (161.

In [ ]:
import numpy as np
from scipy import stats

def student_confidence_intervals_area_based(
        true_intervals_list,       # список списков «истинных» интервалов по изображениям
        pred_intervals_list,       # список списков «предсказанных» интервалов по изображениям
        merge_gap_true=0.5,        # порог объединения для true
        merge_gap_pred=0.5,        # порог объединения для pred
        alpha=0.05                 # уровень значимости: 95%-интервал по умолчанию
    ):
    """
    Строит t-интервалы (Student’s criterion) для каждого из area-based метрик (precision, recall, f1, IoU).
    1. Для каждого изображения рассчитывает значение метрик.
    2. Для каждого вектора значений метрик строит CI:
       mean ± t_{1-α/2, df=n-1} * (sd / sqrt(n))
    Возвращает словарь {метрика: (lower, upper)}.
    """
    # вспомогательная функция, рассчитывающая метрики по одному изображению
    def metrics_one(true_intervals, pred_intervals):
        # здесь используете уже имеющиеся у вас функции:
        # merge_intervals_with_gap, compute_area_precision, compute_area_recall, ...
        t_int = merge_intervals_with_gap(true_intervals, merge_gap_true)
        p_int = merge_intervals_with_gap(pred_intervals, merge_gap_pred)

        # Например:
        area_intersection = get_intersection_of_merged_intervals(t_int, p_int)
        area_true = sum(e - s for s, e in t_int)
        area_pred = sum(e - s for s, e in p_int)

        precision = area_intersection / area_pred if area_pred > 0 else 0.0
        recall    = area_intersection / area_true if area_true > 0 else 0.0
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0.0
        iou = area_intersection / (area_true + area_pred - area_intersection) if (area_true + area_pred - area_intersection) > 0 else 0.0

        return precision, recall, f1, iou

    # Собираем по всем изображениям
    metrics_vals = {'precision': [], 'recall': [], 'f1': [], 'iou': []}
    for t_ints, p_ints in zip(true_intervals_list, pred_intervals_list):
        prec, rec, f1, iou = metrics_one(t_ints, p_ints)
        metrics_vals['precision'].append(prec)
        metrics_vals['recall'].append(rec)
        metrics_vals['f1'].append(f1)
        metrics_vals['iou'].append(iou)

    n = len(true_intervals_list)
    df = n - 1
    t_crit = stats.t.ppf(1 - alpha/2, df)

    ci_dict = {}
    for metric, vals in metrics_vals.items():
        arr = np.array(vals)
        mean = arr.mean()
        sem = arr.std(ddof=1) / np.sqrt(n)
        lower = mean - t_crit * sem
        upper = mean + t_crit * sem
        ci_dict[metric] = (lower, upper)

    return ci_dict

# Пример использования:
ci_student = student_confidence_intervals_area_based(
    true_intervals_list=true_intervals,
    pred_intervals_list=predicted_intervals,
    merge_gap_true=merge_gap_for_true_area,
    merge_gap_pred=merge_gap_for_pred_area,
    alpha=0.05
)

print("Доверительные интервалы (Student’s t, 95%):")
for m, (l, u) in ci_student.items():
    print(f"{m}: [{l:.4f}, {u:.4f}]")
